<a href="https://colab.research.google.com/github/pranshudiwan/NLP_CS_6200/blob/main/NLP_doc2vec%2Bwordcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
import pandas as pd
import numpy as np


In [2]:
# Import train data
url_train = 'https://raw.githubusercontent.com/pranshudiwan/NLP_CS_6200/main/train.csv'
train = pd.read_csv(url_train)

# Import est data
url_test = 'https://raw.githubusercontent.com/pranshudiwan/NLP_CS_6200/main/test.csv'
test = pd.read_csv(url_test)

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Thus we have some null values in keyword and location columns. 

In [6]:
train.sample(10)

,id,keyword,location,text,target
3530,5046,eyewitness,UK,RT patrickjbutler: Excellent damiengayle eyewi...,1
6613,9469,terrorism,"Reston, VA, USA",A good piece on Israeli incitement and Jewish ...,1
2590,3718,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
5542,7907,radiation%20emergency,NaN,Who Else Wants Documents Radiation Emergency P...,0
5181,7393,obliterate,London,'We are now prepared to obliterate more rapidl...,0
5050,7198,natural%20disaster,Los Angeles,On the sneak America has us spoiled. A natural...,1
2965,4260,drowning,NaN,'Drowning' - Acrylic 08.05.15 https://t.co/X17...,1
6975,10005,tsunami,NaN,All of this energy,0
7335,10500,wildfire,Worldwide,California wildfire destroys more homes but cr...,1
6628,9492,terrorist,NaN,Captured terrorist Naved not registered as our...,1


In [7]:
import re
train['text'] = train['text'].map(lambda x: re.sub(r'\W+', ' ', x))

In [8]:
train.sample(5)

,id,keyword,location,text,target
5146,7338,nuclear%20reactor,NaN,Finnish ministers Fennovoima nuclear reactor w...,0
3696,5259,fatality,Honduras,Fatality,0
5712,8150,rescuers,Africa,Durban_Knight Rescuers are searching for hund...,1
1386,1999,bush%20fires,NaN,Ted Cruz fires back at Jeb amp Bush ÛÏWe lose ...,0
7065,10119,upheaval,"Auckland, New Zealand",An indepth look at the new world of work and h...,0


In [9]:
train = train.apply(lambda x: x.astype(str).str.lower())

In [10]:
train.sample(5)

,id,keyword,location,text,target
4656,6621,inundated,singapore,myanmar s president urged people to leave a lo...,1
1214,1752,buildings%20burning,"epic city, bb.",i pledge allegiance to the p o p e and the bur...,0
1845,2655,crush,kaneohe,kuualohax more like you love your husband but...,0
7483,10705,wreck,"sf bay area, california / greater phoenix, az",anyone know if fox ûïnews û will be live strea...,0
1001,1453,body%20bagging,nan,i m not a drake fan but i enjoy seeing him bod...,0


In [11]:
train['keyword'] = train['keyword'].map(lambda x: re.sub(r'\W+', ' ', x))
train['location'] = train['location'].map(lambda x: re.sub(r'\W+', ' ', x))
train.sample(5)

,id,keyword,location,text,target
5672,8095,rescued,nan,britons rescued amid himalaya floods http t co...,1
2719,3905,devastated,republica dominicana,losdelsonido obama declares disaster for typh...,1
3739,5315,fire,sì o paulo,marquei como visto game of thrones 3x5 kissed ...,0
6209,8859,smoke,soufside,niggas selling weed just so they can smoke amp...,0
1009,1464,body 20bagging,nan,mzgraciebaby for the record im jumpin out the...,0


In [12]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

train['text']=train['text'].apply(lambda x: remove_emoji(x))
train.sample(5)

,id,keyword,location,text,target
6102,8712,sinking,nan,the sinking ship sinkingshipindy scarlet lane ...,0
2928,4209,drowned,nan,abcnews24 peterdutton_mp he also told you no ...,0
3345,4789,evacuated,benton city washington,our thoughts are with these local residents ti...,1
4775,6794,lightning,state college pa,check out this amazing footage of lightning fi...,0
4433,6306,hostage,,who s that shadow holdin me hostage i ve been ...,0


In [13]:
!pip install pyspellchecker

     |████████████████████████████████| 2.7MB 5.7MB/s 


In [14]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

#train['text']=train['text'].apply(lambda x : correct_spellings(x))
#train.sample(5)

In [15]:
text = 'corrrect me pleas. MY NLP preject is due tomrow'
correct_spellings(text)

'correct me please MY NLP project is due tomorow'

In [16]:
## Replacing common acronyms

def other_clean(text):
        """
            Other manual text cleaning techniques
        """
        # Typos, slang and other
        sample_typos_slang = {
                                "w/e": "whatever",
                                "usagov": "usa government",
                                "recentlu": "recently",
                                "ph0tos": "photos",
                                "amirite": "am i right",
                                "exp0sed": "exposed",
                                "<3": "love",
                                "luv": "love",
                                "amageddon": "armageddon",
                                "trfc": "traffic",
                                "16yr": "16 year"
                                }

        # Acronyms
        sample_acronyms =  { 
                            "mh370": "malaysia airlines flight 370",
                            "okwx": "oklahoma city weather",
                            "arwx": "arkansas weather",    
                            "gawx": "georgia weather",  
                            "scwx": "south carolina weather",  
                            "cawx": "california weather",
                            "tnwx": "tennessee weather",
                            "azwx": "arizona weather",  
                            "alwx": "alabama weather",
                            "usnwsgov": "united states national weather service",
                            "2mw": "tomorrow"
                            }

        
        # Some common abbreviations 
        sample_abbr = {
                        "$" : " dollar ",
                        "€" : " euro ",
                        "4ao" : "for adults only",
                        "a.m" : "before midday",
                        "a3" : "anytime anywhere anyplace",
                        "aamof" : "as a matter of fact",
                        "acct" : "account",
                        "adih" : "another day in hell",
                        "afaic" : "as far as i am concerned",
                        "afaict" : "as far as i can tell",
                        "afaik" : "as far as i know",
                        "afair" : "as far as i remember",
                        "afk" : "away from keyboard",
                        "app" : "application",
                        "approx" : "approximately",
                        "apps" : "applications",
                        "asap" : "as soon as possible",
                        "asl" : "age, sex, location",
                        "atk" : "at the keyboard",
                        "ave." : "avenue",
                        "aymm" : "are you my mother",
                        "ayor" : "at your own risk", 
                        "b&b" : "bed and breakfast",
                        "b+b" : "bed and breakfast",
                        "b.c" : "before christ",
                        "b2b" : "business to business",
                        "b2c" : "business to customer",
                        "b4" : "before",
                        "b4n" : "bye for now",
                        "b@u" : "back at you",
                        "bae" : "before anyone else",
                        "bak" : "back at keyboard",
                        "bbbg" : "bye bye be good",
                        "bbc" : "british broadcasting corporation",
                        "bbias" : "be back in a second",
                        "bbl" : "be back later",
                        "bbs" : "be back soon",
                        "be4" : "before",
                        "bfn" : "bye for now",
                        "blvd" : "boulevard",
                        "bout" : "about",
                        "brb" : "be right back",
                        "bros" : "brothers",
                        "brt" : "be right there",
                        "bsaaw" : "big smile and a wink",
                        "btw" : "by the way",
                        "bwl" : "bursting with laughter",
                        "c/o" : "care of",
                        "cet" : "central european time",
                        "cf" : "compare",
                        "cia" : "central intelligence agency",
                        "csl" : "can not stop laughing",
                        "cu" : "see you",
                        "cul8r" : "see you later",
                        "cv" : "curriculum vitae",
                        "cwot" : "complete waste of time",
                        "cya" : "see you",
                        "cyt" : "see you tomorrow",
                        "dae" : "does anyone else",
                        "dbmib" : "do not bother me i am busy",
                        "diy" : "do it yourself",
                        "dm" : "direct message",
                        "dwh" : "during work hours",
                        "e123" : "easy as one two three",
                        "eet" : "eastern european time",
                        "eg" : "example",
                        "embm" : "early morning business meeting",
                        "encl" : "enclosed",
                        "encl." : "enclosed",
                        "etc" : "and so on",
                        "faq" : "frequently asked questions",
                        "fawc" : "for anyone who cares",
                        "fb" : "facebook",
                        "fc" : "fingers crossed",
                        "fig" : "figure",
                        "fimh" : "forever in my heart", 
                        "ft." : "feet",
                        "ft" : "featuring",
                        "ftl" : "for the loss",
                        "ftw" : "for the win",
                        "fwiw" : "for what it is worth",
                        "fyi" : "for your information",
                        "g9" : "genius",
                        "gahoy" : "get a hold of yourself",
                        "gal" : "get a life",
                        "gcse" : "general certificate of secondary education",
                        "gfn" : "gone for now",
                        "gg" : "good game",
                        "gl" : "good luck",
                        "glhf" : "good luck have fun",
                        "gmt" : "greenwich mean time",
                        "gmta" : "great minds think alike",
                        "gn" : "good night",
                        "g.o.a.t" : "greatest of all time",
                        "goat" : "greatest of all time",
                        "goi" : "get over it",
                        "gps" : "global positioning system",
                        "gr8" : "great",
                        "gratz" : "congratulations",
                        "gyal" : "girl",
                        "h&c" : "hot and cold",
                        "hp" : "horsepower",
                        "hr" : "hour",
                        "hrh" : "his royal highness",
                        "ht" : "height",
                        "ibrb" : "i will be right back",
                        "ic" : "i see",
                        "icq" : "i seek you",
                        "icymi" : "in case you missed it",
                        "idc" : "i do not care",
                        "idgadf" : "i do not give a damn fuck",
                        "idgaf" : "i do not give a fuck",
                        "idk" : "i do not know",
                        "ie" : "that is",
                        "i.e" : "that is",
                        "ifyp" : "i feel your pain",
                        "IG" : "instagram",
                        "iirc" : "if i remember correctly",
                        "ilu" : "i love you",
                        "ily" : "i love you",
                        "imho" : "in my humble opinion",
                        "imo" : "in my opinion",
                        "imu" : "i miss you",
                        "iow" : "in other words",
                        "irl" : "in real life",
                        "j4f" : "just for fun",
                        "jic" : "just in case",
                        "jk" : "just kidding",
                        "jsyk" : "just so you know",
                        "l8r" : "later",
                        "lb" : "pound",
                        "lbs" : "pounds",
                        "ldr" : "long distance relationship",
                        "lmao" : "laugh my ass off",
                        "lmfao" : "laugh my fucking ass off",
                        "lol" : "laughing out loud",
                        "ltd" : "limited",
                        "ltns" : "long time no see",
                        "m8" : "mate",
                        "mf" : "motherfucker",
                        "mfs" : "motherfuckers",
                        "mfw" : "my face when",
                        "mofo" : "motherfucker",
                        "mph" : "miles per hour",
                        "mr" : "mister",
                        "mrw" : "my reaction when",
                        "ms" : "miss",
                        "mte" : "my thoughts exactly",
                        "nagi" : "not a good idea",
                        "nbc" : "national broadcasting company",
                        "nbd" : "not big deal",
                        "nfs" : "not for sale",
                        "ngl" : "not going to lie",
                        "nhs" : "national health service",
                        "nrn" : "no reply necessary",
                        "nsfl" : "not safe for life",
                        "nsfw" : "not safe for work",
                        "nth" : "nice to have",
                        "nvr" : "never",
                        "nyc" : "new york city",
                        "oc" : "original content",
                        "og" : "original",
                        "ohp" : "overhead projector",
                        "oic" : "oh i see",
                        "omdb" : "over my dead body",
                        "omg" : "oh my god",
                        "omw" : "on my way",
                        "p.a" : "per annum",
                        "p.m" : "after midday",
                        "pm" : "prime minister",
                        "poc" : "people of color",
                        "pov" : "point of view",
                        "pp" : "pages",
                        "ppl" : "people",
                        "prw" : "parents are watching",
                        "ps" : "postscript",
                        "pt" : "point",
                        "ptb" : "please text back",
                        "pto" : "please turn over",
                        "qpsa" : "what happens", #"que pasa",
                        "ratchet" : "rude",
                        "rbtl" : "read between the lines",
                        "rlrt" : "real life retweet", 
                        "rofl" : "rolling on the floor laughing",
                        "roflol" : "rolling on the floor laughing out loud",
                        "rotflmao" : "rolling on the floor laughing my ass off",
                        "rt" : "retweet",
                        "ruok" : "are you ok",
                        "sfw" : "safe for work",
                        "sk8" : "skate",
                        "smh" : "shake my head",
                        "sq" : "square",
                        "srsly" : "seriously", 
                        "ssdd" : "same stuff different day",
                        "tbh" : "to be honest",
                        "tbs" : "tablespooful",
                        "tbsp" : "tablespooful",
                        "tfw" : "that feeling when",
                        "thks" : "thank you",
                        "tho" : "though",
                        "thx" : "thank you",
                        "tia" : "thanks in advance",
                        "til" : "today i learned",
                        "tl;dr" : "too long i did not read",
                        "tldr" : "too long i did not read",
                        "tmb" : "tweet me back",
                        "tntl" : "trying not to laugh",
                        "ttyl" : "talk to you later",
                        "u" : "you",
                        "u2" : "you too",
                        "u4e" : "yours for ever",
                        "utc" : "coordinated universal time",
                        "w/" : "with",
                        "w/o" : "without",
                        "w8" : "wait",
                        "wassup" : "what is up",
                        "wb" : "welcome back",
                        "wtf" : "what the fuck",
                        "wtg" : "way to go",
                        "wtpa" : "where the party at",
                        "wuf" : "where are you from",
                        "wuzup" : "what is up",
                        "wywh" : "wish you were here",
                        "yd" : "yard",
                        "ygtr" : "you got that right",
                        "ynk" : "you never know",
                        "zzz" : "sleeping bored and tired"
                        }
            
        sample_typos_slang_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_typos_slang.keys()) + r')(?!\w)')
        sample_acronyms_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_acronyms.keys()) + r')(?!\w)')
        sample_abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_abbr.keys()) + r')(?!\w)')
        
        text = sample_typos_slang_pattern.sub(lambda x: sample_typos_slang[x.group()], text)
        text = sample_acronyms_pattern.sub(lambda x: sample_acronyms[x.group()], text)
        text = sample_abbr_pattern.sub(lambda x: sample_abbr[x.group()], text)
        
        return text

In [17]:
## !Need to change variable names and abbrevations as appropriate
train["text"] = train["text"].apply(lambda x: other_clean(x))
train.sample(5)

,id,keyword,location,text,target
3328,4768,evacuated,portland oregon,red cross re opens shelter at bickleton school...,1
5294,7561,outbreak,united states,families to sue over legionnaires more than 40...,1
907,1312,bloody,singapore,eh hello cover your bloody thighs your bloody ...,0
3266,4691,engulfed,nan,he came to a land which was engulfed in tribal...,0
1853,2664,crush,nan,ina buted girl crush,1


In [18]:
## Removing single characters
train["text"] = train["text"].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
def tokenizer(text):
    tokenized = []
    for string in text:
        string = re.sub('[^a-z\sA-Z]', '', string)
        # get rid of all the http syntax
        string = re.sub('http\S+', '', string)
        tokenized.append([w for w in string.split() if w not in stop])
    return tokenized

In [21]:
train['tokenized'] = tokenizer(train["text"])

In [22]:
train.sample(5)

,id,keyword,location,text,target,tokenized
3262,4687,engulfed,london,suelinflower there is no words to describe th...,0,"[suelinflower, words, describe, physical, pain..."
4611,6554,injury,nan,dante exum knee injury could stem jazz hoped f...,1,"[dante, exum, knee, injury, could, stem, jazz,..."
1080,1560,bomb,lagos nigeria,if fall is men god praiz8 is bomb well av alwa...,1,"[fall, men, god, praiz, bomb, well, av, always..."
7423,10620,wounded,nan,police officer wounded suspect dead after exch...,1,"[police, officer, wounded, suspect, dead, exch..."
1456,2099,casualty,nan,casualty insurance jobs against hunt up willin...,0,"[casualty, insurance, jobs, hunt, willingheart..."


In [23]:
# Performing stemming

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
def stem(unstemmed_list):  
  ps = PorterStemmer()

  final_stemmed_list = []

  for i in range(len(unstemmed_list)):
    sentence = unstemmed_list[i]
    #print(sentence)
    words = word_tokenize(sentence)
    stemmed_list = []
    for w in words:
        stemmed_list.append(ps.stem(w))
    stemmed_list = ' '.join(stemmed_list)
    final_stemmed_list.append(stemmed_list)

  return final_stemmed_list


In [26]:
unstemmed_list = train['text'].tolist()
train['stemmed_text'] = stem(unstemmed_list)


In [27]:
#Removing digits
train['stemmed_text'] = train['stemmed_text'].str.replace('\d+', '')

In [28]:
train.sample(5)

,id,keyword,location,text,target,tokenized,stemmed_text
6610,9466,terrorism,nan,dhs refuses to call chattanooga islamic terror...,1,"[dhs, refuses, call, chattanooga, islamic, ter...",dh refus to call chattanooga islam terror ûª o...
2849,4095,displaced,palestine,cityofhummus ilnewsflash do you want to hear ...,0,"[cityofhummus, ilnewsflash, want, hear, displa...",cityofhummu ilnewsflash do you want to hear mo...
273,397,apocalypse,sindria,ohh no fukurodani didn survive the apocalypse ...,1,"[ohh, fukurodani, survive, apocalypse, bokuto,...",ohh no fukurodani didn surviv the apocalyps bo...
6346,9075,structural 20failure,nan,investigators have said virgin galactic spaces...,1,"[investigators, said, virgin, galactic, spaces...",investig have said virgin galact spaceship cra...
5729,8176,rescuers,washington,news many deaths in shipwreck rescuers are tr...,1,"[news, many, deaths, shipwreck, rescuers, tryi...",news mani death in shipwreck rescuer are tri t...


In [29]:
# Data visualization imports
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

# importing images
from google.colab import files
from IPython.display import Image


In [30]:
twitter_image = files.upload()

KeyboardInterrupt: ignored

In [31]:
# word cloud of keywords for both classes (starts)
disaster_tweets = train.groupby("target")

disaster_tweets.describe().head()

id         ...                                       stemmed_text     
       count unique  ...                                                top freq
target               ...                                                        
0       4342   4342  ...  black eye space battl occur at star o involv f...    7
1       3271   3271  ...   year old boy charg with manslaught of toddler...   10

[2 rows x 24 columns]

In [32]:
# divide disaster tweets by text
disaster_tweets = train[train['target'] == '1'].text
disaster_tweets = " ".join(line for line in disaster_tweets)

non_disaster_tweets = train[train['target'] == '0'].text
non_disaster_tweets = " ".join(line for line in non_disaster_tweets)

In [33]:
# Setting mask for wordcloud.
from PIL import Image
twitter_mask = np.array(Image.open('twitter_mask.png'))

FileNotFoundError: ignored

In [ ]:
# Function: plot_wordcloud
# Params: title (str) - title for the wordcloud,
#         text (str) - joined text to display
#         background_color (str) - "white", "black"
# Does: plots the wordcloud
def plot_wordcloud(title, text, background_color):
  stopwords = set(STOPWORDS)
  # update stowords (if needed)
  stopwords.update(["https", "co", "__", "amp", "û", "will", "new", "I"])


  wordcloud = WordCloud(mask=twitter_mask, stopwords=stopwords, background_color=background_color,
                        width=1800, height=1400).generate(text)
  plt.figure(figsize=(20,10))
  plt.imshow(wordcloud, interpolation="bilinear")
  plt.axis("off")
  plt.tight_layout(pad=0)

  plt.title(title,
              fontdict={
                  'size': 20,
                  'verticalalignment': 'bottom'
              })
  plt.show()

In [ ]:
plot_wordcloud("Most common words in disaster tweets", disaster_tweets, "black")

In [ ]:
plot_wordcloud("Most common words in non-disaster tweets", non_disaster_tweets, "white")

disaster tweets have much more specific terms like "malaysian airlines", "norther california". 

non-disaster tweet terms are more general like "people", "day", "game"..


Some common words involved links (which is added in stopwords becauset they are irrelevant).
Still looking for ways to get rid of "__" and "u hat".


Doc2Vec build vector embeddings for each document in data. 
(developed by same creators of word2vec)


In [88]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm 

In [89]:

# Function: classify_doc2vec
# Params: train and test dataframe (each with "stemmed_text" and "target" column)
#         n - (odd integer) number of most similar vectors for classification
# Returns: classification of test dataset (list)
# Does: classify all the test tweets based on doc2vec modeling
#       finds n most similar documents from train dataset and classify based on majority
def classify_doc2vec(train, test, n):
  # prepare all the tokens and classifications fror modeling 
  test_x = list(tokenizer(test["stemmed_text"]))
  train_x = list(tokenizer(train["stemmed_text"]))
  train_y = list(train["target"])

  # tag the document and model (use devset to tune the params)
  # model - using distributed bag of words version
  # larger vectors give more accurate results (vector size = 1000, give 70% accuracy)
  tagged_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_x)]
  model = Doc2Vec(tagged_docs, vector_size=1000, dm = 0, window=7, min_count=1, epochs = 100)
  
  # get the most similar vectors for each text in testset and classify them by majority
  # ex. if 3/5 similar vectors were classified as disaster tweet, that tweet is a disaster tweets
  
  pred_y = []
  for text in tqdm(test_x):
    v = model.infer_vector(text, steps = 100)
    similar_vectors = model.docvecs.most_similar([v], topn = n)

    class_dic = {'1': 0, '0': 0} 
    for i, j in similar_vectors:
      pred = train_y[i]
      class_dic[pred] += 1 
    
    if (class_dic['1'] > class_dic['0']):
      pred_y.append('1')
    else:
      pred_y.append('0')

  return pred_y

# Function: evaluate pred
# params: expected (correct classification), real (prediction)
#         both should be 'list' type and 
#         elements should be string (so '1' not integer 1)
# Returns: dictionary of accuracy, precision, recall, and f1
# followed formulas here: https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
def evaluate_pred(expected, pred):
  # true positive (TP): expected = '1', pred = '1'
  # false negative (FN): expected = '1', pred = '0'
  # false positive (FP): expected = '0', pred = '1'
  # true negative (TN): expected = '0', pred = '0'
  TP = 0
  FN = 0
  FP = 0
  TN = 0

  if len(pred) != len(expected):
    print("The number of predictions are not equal to number of expected")
    assert 1 == 2

  for i in range(len(pred)):
    e = expected[i]
    p = pred[i]
    if e == '1':
      if p == '1':
        TP += 1
      else:
        FN += 1
    else:
      if p == '1':
        FP += 1
      else:
        TN += 1

  # accuracy = correct predictions over total (disaster tweets + non-disaster tweets)
  # Precision = correct disaster predictions / total predicted disaster tweets
  # Recall = correct disaster predictions / total actual disaster tweets 
  # F1 = harmonic mean of precision and recall 
  accuracy = (TP + TN) / len(pred)
  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  F1 = 2 * (recall * precision) / (recall + precision)

  return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': F1}




In [58]:
# test on dev set 
dev_set = train.iloc[:1500]
train_set = train.iloc[1500:]

# changing numbers of topn didn't change accuracy that much
# greater vector size and epoch affected accuracy way more
# generally better with recall than precision 
pred = classify_doc2vec(train_set, dev_set, 5)
evaluate_pred(list(dev_set['target']), pred)

100%|██████████| 1500/1500 [00:16<00:00, 91.79it/s]


{'F1': 0.6321243523316062,
 'accuracy': 0.716,
 'precision': 0.6110183639398998,
 'recall': 0.6547406082289803}

In [78]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

true_y = list(dev_set['target'])
print('F1 = ', f1_score(y_true = true_y, y_pred = pred, pos_label="1"))
print('Accuracy = ', precision_score(y_true = true_y, y_pred = pred, pos_label="1"))

F1 =  0.6321243523316062
Accuracy =  0.6110183639398998


Try SVC modeling (support vector classifier). Fit data we provide, returning best fit"

In [91]:
train['tokenized_stem'] = tokenizer(train["stemmed_text"])

val_set = train.iloc[:1500]
train_set = train.iloc[1500:]

val_x = list(val_set["tokenized_stem"])
val_y = list(val_set["target"])

train_x = list(train_set["tokenized_stem"])
train_y = list(train_set["target"])

In [95]:

tagged_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_x)]
model = Doc2Vec(tagged_docs, vector_size=200, dm = 0, window=7, min_count=1, epochs = 40)

train_vec = []
val_vec = []
for text in train_x:
  train_vec.append(model.infer_vector(text, steps = 20))

for text in val_x:
  val_vec.append(model.infer_vector(text, steps = 20))



In [96]:
from sklearn.svm import SVC

svc = SVC(random_state=1, C = 5, gamma = 6, kernel = 'rbf', probability = True)
svc.fit(train_vec, train_y)

svc_pred = svc.predict(val_vec)

print('F1 = ', f1_score(y_true = val_y, y_pred = svc_pred, pos_label="1"))
print('Accuracy = ', precision_score(y_true = val_y, y_pred = svc_pred, pos_label="1"))

F1 =  0.059027777777777776
Accuracy =  1.0


In [94]:
# better with precision, less accurate with recall (vector-200)
evaluate_pred(true_y, svc_pred)

{'F1': 0.5782312925170068,
 'accuracy': 0.752,
 'precision': 0.7894736842105263,
 'recall': 0.4561717352415027}

In [97]:
# better with precision, less accurate with recall (vector - 1000)
# more accurate precision, but hurts recall 
# works better with smaller vectors 
evaluate_pred(true_y, svc_pred)

{'F1': 0.059027777777777776,
 'accuracy': 0.6386666666666667,
 'precision': 1.0,
 'recall': 0.03041144901610018}

In [ ]:
# predict on test set (prep)
# unstemmed_list = test['text'].tolist()
# test['stemmed_text'] = stem(unstemmed_list)
# test['stemmed_text'] = test['stemmed_text'].str.replace('\d+', '')




Viz Ideas:
1. Class Distribution - Sindhu
2. Word cloud of keywords for both classes - Hearan
3. Bi-gram bar plot (bar plot of most frequently occuring words for both classes) - Pranshu
4. Average length of tweet for both classes - Sindhu
5. 

Data processing proposed steps: (Done)
1. Removal of punctuations, unwanted text
2. Performing stemming/lemmatization
3. Spell check
4. Removing emojis
5. Replacing common acronyms/shorhands
6.


EDA Ideas:
1. Clustering Tweets from same location over a short time period - (if many tweets come from the same location in a short time containing keywords for disasters they might actually be a disaster)
2. Clustering tweets via keyword - if the same keyword is being used over a short period of time it might mean a disaster
3. 

Models:
1. Word2Vec with GloVe
2. Simple LSTMs
3. LSTMs with Hidden layers
4. (Possibly) BERT



(Rough sketch)

Project doc: (numbers denote page numbers)

1. Names, Project title)
2. Introduction:
What is the problem we're trying to solve, How are we using NLP to solve the problem, Proposed method (in brief) on how we'll solve, Impact of problem on society (use-case irl)
3. Dataset used, data cleaning methods (and why it was necessary)
4. EDA: 
5. Visualizations
6. Models used (1 page each per model):
Explaining the model architecture in brief, why was it best to use this model (or appropriate), what was the model parameters (explain a few if needed), what were the results, and what metrics were used to obtain the resutls
9. Conclusions, model comparisons, final words
10. Team member contributions







